<a href="https://colab.research.google.com/github/pgurazada/causal_inference/blob/master/01_ladders_of_causality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install -q minepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.0/497.0 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for minepy: filename=minepy-1.2.6-cp310-cp310-linux_x86_64.whl size=187006 sha256=603059c9cf33e51fc24aa46c476c2de815c9354d535267891f83a38d01e6e30c
  Stored in directory: /root/.cache/pip/wheels/69/38/a6/825bb9b9ed81e6af43a0ef80c7cfe4cafcfdbc2f5cde2959d9
Successfully built minepy


In [27]:
import numpy as np
from scipy import stats
from minepy import MINE

# Association

In [3]:
class BookSCM:
    def __init__(self, random_seed=42):
        """
        We assume that the unobserved noise variables
        u_0 and u_1 are uncorrelated
        """
        self.random_seed = random_seed
        self.u_0 = stats.uniform()
        self.u_1 = stats.norm()

    def sample(self, sample_size=100):
        """Samples from the SCM"""
        if self.random_seed:
            np.random.seed(self.random_seed)

        u_0 = self.u_0.rvs(sample_size)
        u_1 = self.u_1.rvs(sample_size)
        a = u_0 > .61
        b = (a + .5 * u_1) > .2

        return a, b

In [5]:
scm = BookSCM()

buy_book_a, buy_book_b = scm.sample(100)

In [19]:
buy_book_a.shape, buy_book_b.shape

((100,), (100,))

In [9]:
prob_book_a = buy_book_a.sum() / buy_book_a.shape[0]
prob_book_b = buy_book_b.sum() / buy_book_b.shape[0]

In [10]:
prob_book_a, prob_book_b

(0.35, 0.55)

In [11]:
prob_book_a_given_book_b = np.where(buy_book_b, buy_book_a, 0).sum() / buy_book_b.sum()
prob_book_b_given_book_a = np.where(buy_book_a, buy_book_b, 0).sum() / buy_book_a.sum()

In [12]:
prob_book_a_given_book_b, prob_book_b_given_book_a

(0.6181818181818182, 0.9714285714285714)

In [22]:
np.where(buy_book_b, buy_book_a, 0)

array([0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0])

# Intervention

In [3]:
np.random.seed(45)
SAMPLE_SIZE = 100

In [4]:
u_0 = np.random.randn(SAMPLE_SIZE)
u_1 = np.random.randn(SAMPLE_SIZE)
a = u_0
b = 5 * a + u_1

r, p = stats.pearsonr(a, b)
r, p

(0.9776350497880796, 3.5878300257941764e-68)

In [5]:
print(f'Mean of B before any intervention: {b.mean():.3f}')
print(f'Variance of B before any intervention: {b.var():.3f}')
print(f'Correlation between A and B:\nr = {r:.3f}; p = {p:.3f}\n')

a = np.array([1.5] * SAMPLE_SIZE)
b = 5 * a + u_1

print(f'Mean of B after the intervention on A: {b.mean():.3f}')
print(f'Variance of B after the intervention on A: {b.var():.3f}\n')

Mean of B before any intervention: -0.620
Variance of B before any intervention: 22.667
Correlation between A and B:
r = 0.978; p = 0.000

Mean of B after the intervention on A: 7.575
Variance of B after the intervention on A: 1.003



In [6]:
a = u_0
b = np.random.randn(SAMPLE_SIZE)

r, p = stats.pearsonr(a, b)

print(f'Mean of B after the intervention on B: {b.mean():.3f}')
print(f'Variance of B after the intervention on B: {b.var():.3f}')
print(f'Correlation between A and B after intervening on B:\nr = {r:.3f}; p = {p:.3f}\n')

Mean of B after the intervention on B: 0.186
Variance of B after the intervention on B: 0.995
Correlation between A and B after intervening on B:
r = -0.023; p = 0.821



When there is causation but no correlation (in the monotonic sense).

A simple case is non-linear relationships between $x$ and $y$.

In [24]:
x = stats.uniform(loc=-2, scale=4).rvs(SAMPLE_SIZE)
y = x**2 + 0.2 * np.random.randn(SAMPLE_SIZE)

In [25]:
r, p = stats.pearsonr(x, y)
r, p

(-0.16984207823854364, 0.09114954756216956)

But $x$ causes y here. We will need a measure such as Maximal Information Coefficient (MIC) that can sniff out non-monotonic correlations.

In [28]:
mine = MINE(alpha=0.6, c=15, est="mic_approx")

In [29]:
mine.compute_score(x, y)

In [30]:
mine.mic()

0.9395815602003367

Bad sampling

In [35]:
# Get initial x
x = np.random.uniform(-2, 2, 5000)

# Filter samples
x = x[np.where((x < -1.9) | (x > 1.9), True, False)]

# Get y
y = x**2 + 0.2*np.random.randn(len(x))

In [36]:
mine.compute_score(x, y)

In [37]:
mine.mic()

0.2614719216561705

In [38]:
r, p = stats.pearsonr(x, y)
r, p

(0.09435941057381711, 0.1433155258322966)

# Counterfactuals

Given the following deterministic SCM:

$
Y = TU + (T-1)(U-1)
$

In [31]:
class CounterfactualSCM:

    def abduct(self, t, y):
        return (t + y - 1)/(2*t - 1)

    def modify(self, t):
        return lambda u: t * u + (t - 1) * (u - 1)

    def predict(self, u, t):
        return self.modify(t)(u)

In [32]:
t, y = 1, 1
coffee = CounterfactualSCM()

In [33]:
u = coffee.abduct(t, y)

In [34]:
coffee.predict(u, 0)

0.0